In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras as keras
import time

2024-07-04 16:44:32.858862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, accuracy_score
#from surprise import Dataset, Reader, KNNBasic, SVD

In [3]:
df = pd.read_csv('./Updated_Cleaned_Product_and_Customer_Data - Updated_Cleaned_Product_and_Customer_Data (2).csv')
df['unique_id'] = df.apply(lambda row: '-'.join([
    row['theme'],
    row['color'],
    str(row['ASD Friendly']),
    str(row['ADHD Friendly']),
    str(row['Visual Impairments'])
]), axis=1)
df['new_product_id'], _ = pd.factorize(df['unique_id'])
df.drop('unique_id', axis=1, inplace=True)
#numeric_features = ['year', 'pieces', 'minifigs', 'agerange_min', 'US_retailPrice', 'age', 'max price']
#categorical_features = ['theme', 'color', 'ASD Friendly', 'ADHD Friendly', 'Visual Impairments', 'Special Needs']
#categorical_features = ['color', 'ASD Friendly', 'ADHD Friendly', 'Visual Impairments', 'Special Needs']
numeric_features = ['pieces', 'minifigs', 'agerange_min', 'US_retailPrice', 'age', 'max price']
categorical_features = ['Special Needs', 'color', 'theme']
df

,Id,name,year,theme,pieces,minifigs,agerange_min,US_retailPrice,color,ASD Friendly,ADHD Friendly,Visual Impairments,age,Special Needs,max price,new_product_id
0,5158,Las Vegas,2018,Architecture,487,0,12,47.99,Soft,1,0,0,13,ASD,50,0
1,5159,Shanghai,2018,Architecture,597,0,7,59.99,Neutral,0,0,0,8,No,100,1
2,5160,Great Wall of China,2018,Architecture,551,0,7,49.99,Neutral,0,0,0,10,ASD,50,1
3,5161,Statue of Liberty,2018,Architecture,1685,0,11,119.99,Soft,1,0,0,12,ASD,150,0
4,5162,Las Vegas,2018,Architecture,501,0,7,39.99,Soft,1,0,0,7,ASD,50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3698,6584,Botanical Accessories,2020,Xtra,32,0,3,3.99,Neutral,0,1,0,2,ADHD,50,191
3699,7046,Road Tape,2020,Xtra,35,0,3,8.99,Soft,1,1,0,3,No,50,190
3700,7050,Water Tape,2020,Xtra,23,0,3,8.99,Neutral,0,1,0,4,ADHD,50,191
3701,7317,Chinatown,2021,Xtra,35,0,3,3.99,Neutral,0,1,0,2,No,50,191


In [4]:
df.head()

,Id,name,year,theme,pieces,minifigs,agerange_min,US_retailPrice,color,ASD Friendly,ADHD Friendly,Visual Impairments,age,Special Needs,max price,new_product_id
0,5158,Las Vegas,2018,Architecture,487,0,12,47.99,Soft,1,0,0,13,ASD,50,0
1,5159,Shanghai,2018,Architecture,597,0,7,59.99,Neutral,0,0,0,8,No,100,1
2,5160,Great Wall of China,2018,Architecture,551,0,7,49.99,Neutral,0,0,0,10,ASD,50,1
3,5161,Statue of Liberty,2018,Architecture,1685,0,11,119.99,Soft,1,0,0,12,ASD,150,0
4,5162,Las Vegas,2018,Architecture,501,0,7,39.99,Soft,1,0,0,7,ASD,50,0


In [5]:
encoder = LabelEncoder()
df['name'] = encoder.fit_transform(df['name'])
df['theme'] = encoder.fit_transform(df['theme'])
df['color'] = encoder.fit_transform(df['color'])
df['Special Needs'] = encoder.fit_transform(df['Special Needs'])

In [6]:
correlation_matrix = df.corr()
correlation_matrix

,Id,name,year,theme,pieces,minifigs,agerange_min,US_retailPrice,color,ASD Friendly,ADHD Friendly,Visual Impairments,age,Special Needs,max price,new_product_id
Id,1.000000,-0.056653,0.978839,0.009700,0.032298,-0.054820,0.125817,5.062035e-02,-0.029932,-0.031082,-0.159334,-5.335373e-02,0.107137,0.048864,0.029050,0.016657
name,-0.056653,1.000000,-0.036416,0.033337,0.030551,0.064147,0.024816,3.443808e-02,-0.028787,-0.002533,0.045190,6.385992e-02,0.024548,0.017893,0.034972,0.032287
year,0.978839,-0.036416,1.000000,-0.018088,0.066077,-0.030038,0.143385,7.408462e-02,-0.025110,-0.019662,-0.119516,-1.291435e-02,0.125122,0.043603,0.048083,-0.014348
theme,0.009700,0.033337,-0.018088,1.000000,0.068793,0.070120,0.117173,9.880145e-02,0.024786,-0.001225,-0.091035,-1.145186e-01,0.102371,0.008485,0.072545,0.998977
pieces,0.032298,0.030551,0.066077,0.068793,1.000000,0.342500,0.665468,8.709058e-01,0.013610,0.018495,-0.226338,-3.949754e-02,0.607693,0.055841,0.747247,0.073479
minifigs,-0.054820,0.064147,-0.030038,0.070120,0.342500,1.000000,0.190494,4.022918e-01,-0.010257,0.018039,-0.121306,4.211897e-02,0.164116,0.058668,0.327196,0.068839
agerange_min,0.125817,0.024816,0.143385,0.117173,0.665468,0.190494,1.000000,6.036118e-01,0.027863,0.030147,-0.532687,-1.052416e-01,0.911986,0.165264,0.496574,0.126704
US_retailPrice,0.050620,0.034438,0.074085,0.098801,0.870906,0.402292,0.603612,1.000000e+00,0.006323,0.015225,-0.207105,-9.204825e-07,0.544371,0.061290,0.854327,0.103540
color,-0.029932,-0.028787,-0.025110,0.024786,0.013610,-0.010257,0.027863,6.323433e-03,1.000000,0.897043,-0.013016,-4.381163e-01,0.031561,-0.258976,-0.006732,0.023837
ASD Friendly,-0.031082,-0.002533,-0.019662,-0.001225,0.018495,0.018039,0.030147,1.522505e-02,0.897043,1.000000,-0.012195,-1.004519e-01,0.036306,-0.196984,-0.005411,-0.001227


In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [8]:
X = preprocessor.fit_transform(df[numeric_features + categorical_features])
encoder = LabelEncoder()
y = encoder.fit_transform(df['new_product_id'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'Accuracy: {accuracy:.2f}')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.85
F1 Score: 0.83


In [19]:
from sklearn.model_selection import cross_val_score, GridSearchCV
k_range = range(1,31)
param_grid = dict(n_neighbors=k_range)
grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')  # 10-fold cross-validation
grid.fit(X_train, y_train)
print('Best k value:', grid.best_params_)
print('Best cross-validated score:', grid.best_score_)

/Users/user/anaconda3/envs/tensorflow/lib/python3.11/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best k value: {'n_neighbors': 1}
Best cross-validated score: 0.8815009571122555


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score

print('Begin Random Forest Implementation...')
# 1. Create the RandomForestClassifier model object below and assign to variable 'rf_20_model'

rf_20_model = RandomForestClassifier(criterion='entropy', n_estimators=50)

# 2. Fit the model to the training data below

rf_20_model.fit(X_train, y_train)

# 3. Make predictions on the test data using the predict() method and assign the result to a
# list named 'rf_20_predictions' below

rf_20_predictions = rf_20_model.predict(X_test)

# 4. Calculate F1 score and accuracy for rf_20_model

f1_score_20 = f1_score(y_test, rf_20_predictions, average='weighted')
accuracy_score_20 = accuracy_score(y_test, rf_20_predictions)

print(f'F1 Score for 50 estimators: {f1_score_20}')
print(f'Accuracy Score for 50 estimators: {accuracy_score_20}')

print('End')

Begin Random Forest Implementation...
F1 Score for 50 estimators: 0.9062260052772526
Accuracy Score for 50 estimators: 0.9203778677462888
End


In [12]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Assuming X_train, y_train, X_test, and y_test are already defined and preprocessed

# Determine the number of unique classes
num_classes = max(y_train.max(), y_test.max()) + 1  # Ensure we cover all classes
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

# Convert sparse matrix to dense
X_train_dense = X_train.todense()
X_test_dense = X_test.todense()

# Manually split the data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_dense, y_train_categorical, test_size=0.2, random_state=42)

print('Begin Neural Network Implementation...')

# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Use 'sigmoid' for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
model.fit(X_train_split, y_train_split, epochs=20, batch_size=32, validation_data=(X_val_split, y_val_split))

# Make predictions on the test data
y_pred_prob = model.predict(X_test_dense)
y_pred = y_pred_prob.argmax(axis=1)

# Calculate F1 score and accuracy
f1_score_nn = f1_score(y_test, y_pred, average='weighted')
accuracy_score_nn = accuracy_score(y_test, y_pred)

print(f'F1 Score for Neural Network: {f1_score_nn}')
print(f'Accuracy Score for Neural Network: {accuracy_score_nn}')

print('End')


Begin Neural Network Implementation...
Epoch 1/20
75/75 [==============================] - 1s 6ms/step - loss: 4.9234 - accuracy: 0.1389 - val_loss: 4.2331 - val_accuracy: 0.1535
Epoch 2/20
75/75 [==============================] - 0s 3ms/step - loss: 3.6305 - accuracy: 0.2039 - val_loss: 3.3442 - val_accuracy: 0.2968
Epoch 3/20
75/75 [==============================] - 0s 3ms/step - loss: 2.8547 - accuracy: 0.3917 - val_loss: 2.6681 - val_accuracy: 0.4755
Epoch 4/20
75/75 [==============================] - 0s 3ms/step - loss: 2.2081 - accuracy: 0.5555 - val_loss: 2.1080 - val_accuracy: 0.6003
Epoch 5/20
75/75 [==============================] - 0s 3ms/step - loss: 1.6894 - accuracy: 0.6779 - val_loss: 1.6438 - val_accuracy: 0.6830
Epoch 6/20
75/75 [==============================] - 0s 3ms/step - loss: 1.2835 - accuracy: 0.7573 - val_loss: 1.2941 - val_accuracy: 0.7841
Epoch 7/20
75/75 [==============================] - 0s 3ms/step - loss: 0.9719 - accuracy: 0.8274 - val_loss: 1.0477 - va

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score

print('Begin SVM Implementation...')

# 1. Create the SVC model object below and assign to variable 'svm_model'
svm_model = SVC(probability=True)

# 2. Fit the model to the training data below
svm_model.fit(X_train, y_train)

# 3. Make predictions on the test data using the predict() method and assign the result to a
# list named 'svm_predictions' below
svm_predictions = svm_model.predict(X_test)

# 4. Calculate F1 score and accuracy for svm_model
f1_score_svm = f1_score(y_test, svm_predictions, average='weighted')
accuracy_score_svm = accuracy_score(y_test, svm_predictions)

print(f'F1 Score for SVM: {f1_score_svm}')
print(f'Accuracy Score for SVM: {accuracy_score_svm}')

print('End')


Begin SVM Implementation...
F1 Score for SVM: 0.8523120705086511
Accuracy Score for SVM: 0.8771929824561403
End


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

print('Begin Logistic Regression Implementation...')

# 1. Create the LogisticRegression model object below and assign to variable 'log_reg_model'
log_reg_model = LogisticRegression(max_iter=200)

# 2. Fit the model to the training data below
log_reg_model.fit(X_train, y_train)

# 3. Make predictions on the test data using the predict() method and assign the result to a
# list named 'log_reg_predictions' below
log_reg_predictions = log_reg_model.predict(X_test)

# 4. Calculate F1 score and accuracy for log_reg_model
f1_score_log_reg = f1_score(y_test, log_reg_predictions, average='weighted')
accuracy_score_log_reg = accuracy_score(y_test, log_reg_predictions)

print(f'F1 Score for Logistic Regression: {f1_score_log_reg}')
print(f'Accuracy Score for Logistic Regression: {accuracy_score_log_reg}')

print('End')


Begin Logistic Regression Implementation...
F1 Score for Logistic Regression: 0.8828152798803317
Accuracy Score for Logistic Regression: 0.9055330634278003
End


In [23]:
import numpy as np

# Define the special needs categories for encoding
special_needs_categories = {'No': 0, 'ASD': 1, 'ADHD': 2, 'Visual Impairment': 3}

def preprocess_input(age, special_needs, price, pieces, minifigs, agerange_min, us_retail_price, preprocessor):
    # Convert special needs to numerical value
    special_needs_value = special_needs_categories.get(special_needs, 0)
    
    # Create a DataFrame for the input features
    input_features = pd.DataFrame({
        'pieces': [pieces],
        'minifigs': [minifigs],
        'agerange_min': [agerange_min],
        'US_retailPrice': [us_retail_price],
        'age': [age],
        'max price': [price],
        'Special Needs': [special_needs_value],
        'color': [0],  # Default value, this needs to be adjusted accordingly
        'theme': [0]   # Default value, this needs to be adjusted accordingly
    })
    
    # Transform the input features using the preprocessor
    input_data = preprocessor.transform(input_features)
    
    return input_data

def recommend_products(age, special_needs, price, model, preprocessor, product_ids, num_recommendations=3):
    # Use average or median values for the other features
    pieces_avg = df['pieces'].median()
    minifigs_avg = df['minifigs'].median()
    agerange_min_avg = df['agerange_min'].median()
    us_retail_price_avg = df['US_retailPrice'].median()
    
    # Preprocess the input data
    input_data = preprocess_input(age, special_needs, price, pieces_avg, minifigs_avg, agerange_min_avg, us_retail_price_avg, preprocessor)
    
    # Predict probabilities for each product ID
    predictions = model.predict_proba(input_data)
    
    # Get the indices of the top N recommended products
    recommended_indices = predictions[0].argsort()[-num_recommendations:][::-1]
    
    # Map the indices to product IDs
    recommended_product_ids = [product_ids[idx] for idx in recommended_indices]
    
    return recommended_product_ids

# Example usage
age = 10
special_needs = 'No'
price = 10.00

# Assuming `rf_20_model` is your trained Random Forest model and `product_ids` is a list of product IDs
product_ids = list(range(num_classes))  # Example product IDs
recommended_products = recommend_products(age, special_needs, price, rf_20_model, preprocessor, product_ids)

print(f'Recommended Product IDs: {recommended_products}')


Recommended Product IDs: [34, 147, 56]
